In [31]:
from lusidtools.jupyter_tools import toggle_code

"""Business agility

Demonstration of how to migrate your data from one system to another.

Attributes
----------
transactions
holdings
set holdings
instruments
properties
transaction configuration
reconciliations
"""

toggle_code("Toggle Docstring")

## The Challenge

One of the biggest challenges in migrating your investment records from one system to another is in ensuring that your transactions are represented correctly in the new system.

It is critical that the new system can interpret each transaction appropriately and determine what the underlying movements in cash and securities are.

This can be difficult when transactions are given esoteric types such as '1085' or 'STP' (STP is the MT940 SWIFT transaction type for Stamp Duty, but it could just as easily be an acronym for 'Sale Third Party' or 'Sale Transaction Processed'). 

The end result is typically a lot of time spent chasing down the handful of people in the organisation that understand the meaning of each transaction type and then laboriously mapping those to the new system's available types. This is one of the reasons that migrations between systems can take so long. 

## The Solution

Rather than forcing a set of transaction types onto you, LUSID allows you to configure your own set of transaction types. Each type specifies the underlying movements that should occur when a transaction with this type is upserted into LUSID. 

In this notebook you will set up your own transaction type configuration and simulate how you might adjust this during a migration. You will achieve this by:

1) Setting up a Scope to Hold your Portfolio

2) Creating your Portfolio

3) Creating your Instrument Universe

4) Setting your Initial Holdings

5) Getting your Output Transactions

6) Setting your Transaction Type Configuration

7) Loading your Transactions 

8) Retrieving your Holdings

9) Updating your Transaction Type Configuration

10) Adjusting your Holdings with the End of Day Report

11) Performing a Reconciliation

12) Investigating the Reconcilliation Breaks & Updating your Transaction Type Configuration

13) Re-running your Reconciliation

*Run the cell below to import the libraries and authenticate your LUSID client*

In [34]:
# Import LUSID
import lusid
import lusid.models as models
import lusid_sample_data as import_data
from lusidjam import RefreshingToken
from lusidtools.cocoon.transaction_type_upload import upsert_transaction_type_alias

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd
import json
import lusid
import os

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")


print ('LUSID Environment Initialised')
print ('LUSID API Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID API Version:  0.6.11506.0


![Init](img/migration-lusidinit.gif)

### 1) Set up your Scope

You will need to create a name for the scope that you will use for the migration. Read more about scopes in the [LUSID Knowledge Base: Scopes](https://support.lusid.com/what-is-a-scope-in-lusid-and-how-is-it-used).

*Run the cell below to create a name for your scope*

In [35]:
# Generate a unique id for your scope
scope_id = import_data.create_scope_id()
# Set the name of your scope
scope = 'investment-book-of-records-{}'.format(scope_id)
prettyprint.heading('Scope', scope)

Scope: investment-book-of-records-3bf2-7ffd-98a6-44


![Scopes](img/migration-scopes.gif)

### 2) Set up your Portfolio

Now that you have decided on the name for your scope, you can create your portfolio inside this scope. Note that every portfolio can be referenced by a unique code. Read more about portfolios in the [LUSID Knowledge Base: Portfolios](https://support.lusid.com/what-is-2).

For further usage of the create portfolio API call refer to the [LUSID API Docs: Create Portfolio](https://docs.lusid.com/#operation/CreatePortfolio).

Note that when you create the portolio in the cell below you are creating it with a 'created' date of 1052 days ago. This number is rather arbitary, in practice it should be the date the portfolio came into existence regardless of the system you first created it in, read more about the importance of the created date on a portfolio in the [LUSID Knowledge Base: Importance of Portfolio Creation Date](https://support.lusid.com/importance-of-portfolio-creation-date).

*Run the cell below to create your portfolio*

In [36]:
# Set the code of your portfolio
portfolio_code = 'Global-Strategies'

# Set the creation date of your portfolio 
portfolio_creation_date = datetime.now(pytz.UTC) - timedelta(days=1052)

# Build your request to create your portfolio
request = models.CreateTransactionPortfolioRequest(
    display_name='Global Strategies Fund',
    code=portfolio_code,
    base_currency='GBP',
    description=None,
    created=portfolio_creation_date,
    corporate_action_source_id=None,
    accounting_method='AverageCost',
    sub_holding_keys=None,
    properties=None)

# Call LUSID to create your portfolio
response = api_factory.build(lusid.api.TransactionPortfoliosApi).create_portfolio(
    scope=scope,
    create_transaction_portfolio_request=request)

# Pretty print the response
prettyprint.portfolio_response(response)

Portfolio Created
Scope: investment-book-of-records-3bf2-7ffd-98a6-44
Code: Global-Strategies
Portfolio Effective From: 2020-08-03 16:38:35.884666+00:00
Portfolio Created On: 2023-06-21 16:38:39.670283+00:00



![Portfolio](img/migration-portfolio.gif)

### 3) Add your Instrument Universe

Before you can take on any holdings or make any trades you need to ensure that your instrument universe has been populated. In this case you will import your instrument universe from a CSV file. Read more about instruments in LUSID in the [LUSID Knowledge Base: Instruments](https://support.lusid.com/what-is-an-instrument).

*Run the cell below to import your instrument universe*

In [37]:
instrument_universe = pd.read_csv('data/instruments.csv')
instrument_universe.head()

,instrument_name,client_internal,currency,isin,figi,exchange_code,country_issue,ticker,market_sector,security_type,coupon
0,Amazon_Nasdaq_AMZN,imd_34634534,USD,US0231351067,BBG000BVPXP1,UN,united_states_america,AMZN,equity,common_stock,NaN
1,Apple_Nasdaq_AAPL,imd_35345345,USD,US0378331005,BBG000B9XVV8,UN,united_states_america,AAPL,equity,common_stock,NaN
2,BP_LondonStockEx_BP,imd_43535553,GBP,GB0007980591,BBG000C05BD1,LN,united_kingdom,BP/,equity,common_stock,NaN
3,BurfordCapital_LondonStockEx_BUR,imd_43534356,GBP,GG00B4L84979,BBG000PN88Q7,LN,united_kingdom,BUR,equity,common_stock,NaN
4,EKFDiagnostics_LondonStockEx_EKF,imd_34535355,GBP,GB0031509804,BBG000BVNBN3,LN,united_kingdom,EKF,equity,common_stock,NaN


Now that you have the details for your instruments you can go ahead and create an instrument definition for each instrument. These can then be upserted into LUSID. Read about instrument definitions here [LUSID Knowledge Base: What is an Instrument?](https://support.lusid.com/what-is-an-instrument).

As part of this definition you will also attach two alias identifiers to your instruments. Read more about alias identifiers here [LUSID Knowledge Base: What is an Alias Identifier?](https://support.lusid.com/alias-identifiers).

You use an upsert method to add instrument definitions to the instrument universe in LUSID. Read more about the behaviour of the upsert method here [LUSID Knowledge Base: Upsert](https://support.lusid.com/upsert-command).

For further usage of the upsert instruments API call refer to the [LUSID API Docs: Upserting Instruments](https://docs.lusid.com/#operation/UpsertInstruments).

*Run the cell below to upsert your instruments into LUSID*

In [38]:
# Initialise your batch upsert request
batch_upsert_request = {}

# Using your instrument universe create your batch request
for index, instrument in instrument_universe.iterrows():

    # Specify the columns of your identifiers
    identifier_columns = [
            ('isin', 'Isin'), 
            ('figi', 'Figi'), 
            ('ticker', 'Ticker'),
            ('client_internal', 'ClientInternal')
    ]
    
    # Create your identifiers
    identifiers = {}
    for identifier in identifier_columns:
        identifiers[identifier[1]] = models.InstrumentIdValue(
            value=instrument[identifier[0]])
    
    # Build your request and add it to the dictionary
    batch_upsert_request[instrument['instrument_name']] = models.InstrumentDefinition(
        name=instrument['instrument_name'],
        identifiers=identifiers)
    
# Call LUSID to upsert your instrument defintions
instrument_response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments(request_body=batch_upsert_request)

# Pretty print the response
prettyprint.instrument_response(instrument_response)

Instruments Successfully Upserted: 


,Instrument,ClientInternal ID,LUSID Instrument ID
0,Whitebread_LondonStockEx_WTB,imd_35349900,LUID_0000MIP2
1,MicroFocus_LondonStockEx_MCRO,imd_34567338,LUID_0000MIP9
2,UKGiltTreasury_2.0_2025,imd_34534536,LUID_0000MIP3
3,Apple_Nasdaq_AAPL,imd_35345345,LUID_0000MIP0
4,TESCO_LondonStockEx_TSCO,imd_34634673,LUID_0000NA4Q
5,Kingfisher_LondonStockEx_KGF,imd_34535552,LUID_0000MIOZ
6,UKGiltTreasury_3.5_2045,imd_54234532,LUID_0000MIPC
7,Amazon_Nasdaq_AMZN,imd_34634534,LUID_0000MIOX
8,USTreasury_2.00_2021,imd_34535347,LUID_0000MIPE
9,USTreasury_6.875_2025,imd_34534539,LUID_0000MIP1


![Instruments](img/migration-instruments.gif)

### 4) Set your Initial Holdings

Now that you have your instrument universe populated you can load your initial holdings into your portfolio. In this case you will import your holdings from a CSV file. 

*Run the cell below to import your take on balances*

In [39]:
holdings = pd.read_csv('data/demo-holdings.csv')
holdings.head()

,portfolio_code,instrument_name,quantity,price,currency,figi
0,Global-Strategies,GBP_Cash,8336000,1.00,GBP,NaN
1,Global-Strategies,Glencore_LondonStockEx_GLEN,905141,2.76,GBP,BBG001MM1KV4
2,Global-Strategies,Kingfisher_LondonStockEx_KGF,1362038,2.28,GBP,BBG000BKH1W6
3,Global-Strategies,UKGiltTreasury_2.0_2025,405589,106.64,GBP,BBG0088JSC32
4,Global-Strategies,UKGiltTreasury_3.5_2045,266169,134.43,GBP,BBG006N6HZM7


Now that you have imported your holdings you can add them to LUSID. You can do this by setting the holdings on a portfolio. Read more about how making an adjustment or setting the holdings on a portfolio affects it here [LUSID Knowledge Base: The effect of holding adjustments](https://support.lusid.com/how-do-holding-adjustments-affect-a-portfolio).

For further usage of the set holdings API call refer to the [LUSID API Docs: Set Holdings](https://docs.lusid.com/#operation/SetHoldings).

*Run the cell below to upsert your holdings into LUSID*

In [40]:
# Make the holdings effective from two days ago
holdings_effective_date = datetime.now(pytz.UTC) - timedelta(days=2)

# Iterate the portfolios in the holdings CSV, note in this case you only have one
for portfolio in holdings['portfolio_code'].unique():
    # Initialise a list to hold your adjustments
    holding_adjustments = []
    
    # Iterate over the holdings in each portfolio
    for index, holding in holdings.loc[holdings['portfolio_code'] == portfolio].iterrows():
        
        # Set your instrument identifiers based on whether or not instrument is cash
        if 'Cash' in holding['instrument_name']:
            identifier_key = 'Instrument/default/Currency'
            identifer = holding['instrument_name'].split('_')[0]
        else:
            identifier_key = 'Instrument/default/Figi'
            identifer = holding['figi']
            
        # Create your holding adjustment and append it to your list
        holding_adjustments.append(
            models.AdjustHoldingRequest(
                instrument_identifiers={
                    identifier_key: identifer},
                tax_lots=[
                    models.TargetTaxLotRequest(
                        units=holding['quantity'],
                        cost=models.CurrencyAndAmount(
                            amount=holding['quantity'] * holding['price'],
                            currency=holding['currency']),
                        portfolio_cost=holding['quantity'] * holding['price'],
                        price=holding['price'])
                ]
            )
        )
    
    # Call LUSID to set your initial holdings
    response = api_factory.build(lusid.api.TransactionPortfoliosApi).set_holdings(
        scope=scope,
        code=portfolio,
        effective_at=holdings_effective_date,
        adjust_holding_request=holding_adjustments)

    # Pretty print our response from LUSID
    prettyprint.set_holdings_response(response, scope, portfolio)

Holdings Successfully Set for Portfolio
Scope: investment-book-of-records-3bf2-7ffd-98a6-44
Code: Global-Strategies
Holdings Effective From: 2020-08-03 16:38:35.884666+00:00
Holdings Created On: 2023-06-21 16:38:46.758672+00:00



![Instruments](img/migration-holdingsadded.gif)

### 5) Get your Output Transactions

You can see how LUSID has adjusted the holdings of our portfolio to meet our initial holdings by generating the output transactions from the portfolio. You can read more about output transactions here [LUSID Knowledge Base: Output Transactions](https://support.lusid.com/what-is-an-output-transaction-in-lusid).

For further usage of the build transactions API call refer to the [LUSID API Docs: Build Output Transactions](https://docs.lusid.com/#operation/BuildTransactions).

*Run the cell below to generate your output transactions*

In [41]:
# Set our query parameters to build your transactions
query_params = models.TransactionQueryParameters(
    start_date=(holdings_effective_date-timedelta(days=4)).isoformat(),
    end_date=(holdings_effective_date+timedelta(days=4)).isoformat(),
    query_mode='TradeDate',
    show_cancelled_transactions=None)

# Call LUSID to build your output transactions
response = api_factory.build(lusid.api.TransactionPortfoliosApi).build_transactions(
    scope=scope,
    code=portfolio_code,
    property_keys=['Instrument/default/Name'],
    transaction_query_parameters=query_params
)

# Pretty print your output transactions
prettyprint.output_transactions(response, scope, portfolio_code, ['Instrument/default/Name'])

Output Transactions for Portfolio
Scope: investment-book-of-records-3bf2-7ffd-98a6-44
Code: Global-Strategies



,Transaction ID,Transaction Type,Instrument/default/Name,Units,Price,Currency,Transaction Date,Settlement Date,Realised Gain Loss
0,2023-06-19T16:38:45.7175870+00:00,AdjustmentIncrease,GBP,8336000.00,1.00,GBP,2023-06-19 16:38:45.717587+00:00,2023-06-19 16:38:45.717587+00:00,NaN
1,2023-06-19T16:38:45.7175870+00:00,AdjustmentIncrease,Glencore_LondonStockEx_GLEN,905141.00,2.76,GBP,2023-06-19 16:38:45.717587+00:00,2023-06-19 16:38:45.717587+00:00,NaN
2,2023-06-19T16:38:45.7175870+00:00,AdjustmentIncrease,Kingfisher_LondonStockEx_KGF,1362038.00,2.28,GBP,2023-06-19 16:38:45.717587+00:00,2023-06-19 16:38:45.717587+00:00,NaN
3,2023-06-19T16:38:45.7175870+00:00,AdjustmentIncrease,UKGiltTreasury_2.0_2025,405589.00,106.64,GBP,2023-06-19 16:38:45.717587+00:00,2023-06-19 16:38:45.717587+00:00,NaN
4,2023-06-19T16:38:45.7175870+00:00,AdjustmentIncrease,UKGiltTreasury_3.5_2045,266169.00,134.43,GBP,2023-06-19 16:38:45.717587+00:00,2023-06-19 16:38:45.717587+00:00,NaN
5,2023-06-19T16:38:45.7175870+00:00,AdjustmentIncrease,UKGiltTreasury_3.75_2021,661713.00,108.13,GBP,2023-06-19 16:38:45.717587+00:00,2023-06-19 16:38:45.717587+00:00,NaN
6,2023-06-19T16:38:45.7175870+00:00,AdjustmentIncrease,UKGiltTreasury_4.5_2034,77481.00,140.57,GBP,2023-06-19 16:38:45.717587+00:00,2023-06-19 16:38:45.717587+00:00,NaN
7,2023-06-19T16:38:45.7175870+00:00,AdjustmentIncrease,USTreasury_2.00_2021,1440244.00,97.90,USD,2023-06-19 16:38:45.717587+00:00,2023-06-19 16:38:45.717587+00:00,NaN
8,2023-06-19T16:38:45.7175870+00:00,AdjustmentIncrease,BurfordCapital_LondonStockEx_BUR,853486.00,14.06,GBP,2023-06-19 16:38:45.717587+00:00,2023-06-19 16:38:45.717587+00:00,NaN
9,2023-06-19T16:38:45.7175870+00:00,AdjustmentIncrease,EKFDiagnostics_LondonStockEx_EKF,925925.00,0.27,GBP,2023-06-19 16:38:45.717587+00:00,2023-06-19 16:38:45.717587+00:00,NaN


![Output Transactions](img/migration-outputtransactions.gif)

### 6) Set your Transaction Type Configuration

Now that you have set your initial holdings you are ready to take on some transactions. You have made several trades over the last day that you would like to add to LUSID. You will import these from a CSV file. 

*Run the cell below to import your transactions*

In [42]:
daily_transactions = pd.read_csv('data/demo-transactions.csv')
daily_transactions.head()

,portfolio_code,transaction_id,instrument_name,transaction_description,transaction_type,transaction_units,transaction_price,transaction_currency,transaction_strategy,transaction_cost,figi
0,Global-Strategies,tid_329432525234324,Kingfisher_LondonStockEx_KGF,Equity Sale,Sell,325000,2.35,GBP,quantitativeSignal,762125.00,BBG000BKH1W6
1,Global-Strategies,tid_325452342424500,UKGiltTreasury_4.5_2034,Equity Purchase,Buy,10501,140.57,GBP,incomeRequirements,1476146.57,BBG0000D14P3
2,Global-Strategies,tid_234295929052090,UKGiltTreasury_3.75_2021,Equity Purchase,Buy,24000,109.13,GBP,incomeRequirements,2619024.00,BBG001KKJLR4
3,Global-Strategies,tid_234942982496001,USTreasury_2.00_2021,Equity Sale,Sell,57000,97.80,USD,internationalExposure,5574600.00,BBG00FN3B5K8
4,Global-Strategies,tid_121309590059995,BP_LondonStockEx_BP,Equity Purchase,Buy,50000,5.29,GBP,fundamentalAnalysis,264350.00,BBG000C05BD1


Before you can add these trades to LUSID you need to set up a transaction type configuration. This allows the LUSID movement engine to correctly decompose each transaction into its underlying economic movements. Read more about configuring transaction types here [LUSID Knowledge Base: Configuring Transaction Types](https://support.lusid.com/configuring-transaction-types) and movements in LUSID here [LUSID Knowledge Base: Movements](https://support.lusid.com/what-is-a-movement-in-lusid). 

You will import the default LUSID transaction type configuration from a JSON file and then use this to set your configuration inside LUSID.

*Run the cell below to import the LUSID default transaction mapping configuration*

In [43]:
# Import the default transaction type configuration and load it into a dictionary 
default_transaction_mapping=open('data/default_transaction_mapping.json').read()
default_transaction_mapping = json.loads(default_transaction_mapping)
# Pretty print your configuration
pprint.pprint(default_transaction_mapping)

{'transactionConfigRequests': [{'aliases': [{'description': 'Purchase',
                                             'transactionClass': 'Basic',
                                             'transactionGroup': 'default',
                                             'transactionRoles': 'LongLonger',
                                             'type': 'Buy'},
                                            {'description': 'PURCHASE',
                                             'transactionClass': 'Basic',
                                             'transactionGroup': 'alt1',
                                             'transactionRoles': 'LongLonger',
                                             'type': 'BY'}],
                                'movements': [{'direction': 1,
                                               'mappings': [],
                                               'movementTypes': 'StockMovement',
                                               'properties': [],
       

Now that you have imported the default configuration you can build our request to set it inside LUSID.

For further usage of the build transactions API call refer to the [LUSID API Docs: Set Configuration Transaction Types](https://docs.lusid.com/#operation/SetConfigurationTransactionTypes).

*Run the cell below to set the transaction type configuration*

In [44]:
# Initialise your list of configuration requests, one for each transaction type
configuration_requests = []

# Iterate over your configurations in the default mapping
for configuration in default_transaction_mapping['transactionConfigRequests']:
    
    # Initialise your list of aliases for this configuration
    aliases = []
    
    # Iterate over the aliases in the imported config 
    for alias in configuration['aliases']:
        # Append the alias to your list
        aliases.append(
            models.TransactionConfigurationTypeAlias(
                type=alias['type'],
                description=alias['description'],
                transaction_class=alias['transactionClass'],
                transaction_group=alias['transactionGroup'],
                transaction_roles=alias['transactionRoles']))
        
        
    # Initialise your list of movements for this configuration
    movements = []
    
    # Iterate over the movements in the impoted config
    for movement in configuration['movements']:
        
        # Add properties if they exist in the config
        if len (movement['properties']) > 0:
            key = movement['properties'][0]['key']
            value = models.PerpetualProperty(
                key=key,
                value=models.PropertyValue(label_value=movement['properties'][0]['value'])
            )
            properties={key: value}                                    
        else:
            properties=None
            
        # Append the movement to your list
        movements.append(
            models.TransactionConfigurationMovementDataRequest(
                movement_types=movement['movementTypes'],
                side=movement['side'],
                direction=movement['direction'],
                properties=properties,
                mappings=None))
    
    # Build your configuration for this transaction type
    configuration_requests.append(
        models.TransactionConfigurationDataRequest(
        aliases=aliases,
        movements=movements,
        properties=None))

# Call LUSID to set your configuration for our transaction types

upsert_txn_config = upsert_transaction_type_alias(api_factory, new_transaction_config= configuration_requests)

![Instruments](img/migration-transactiontypesconfig.gif)

### 7) Add your Transactions Throughout the Day

Now that you have both imported your transactions and set your transaction type configuration you can upsert your transactions into LUSID. For simplicity each transaction has the same trade and settlement date.

Read more about transactions here [LUSID Knowledge Base: Transactions](https://support.lusid.com/what-is-a-transaction). 

For further usage of the upsert transactions API call refer to the [LUSID API Docs: Upsert Transactions](https://docs.lusid.com/#operation/UpsertTransactions).

*Run the cell below to upsert our transactions into LUSID*

In [45]:
# Set your transaction and settlement dates
transaction_date = datetime.now(pytz.UTC)-timedelta(days=1)
settlement_date = datetime.now(pytz.UTC)+timedelta(days=1)

# Iterate over the portfolios in our transactions file, in this case only one
for portfolio in daily_transactions['portfolio_code'].unique():
    
    # Initialise a list to hold your transaction requests
    transactions = []
    
    # Iterate over the transactions in the portfolio
    for index, transaction in daily_transactions.loc[daily_transactions['portfolio_code'] == portfolio].iterrows():
        
        # Append your request to the list
        transactions.append(models.TransactionRequest(
            transaction_id=transaction['transaction_id'],
            type=transaction['transaction_type'],
            instrument_identifiers={
                'Instrument/default/Figi': transaction['figi']
            },
            transaction_date=transaction_date.isoformat(),
            settlement_date=settlement_date.isoformat(),
            units=transaction['transaction_units'],
            transaction_price=models.TransactionPrice(
                  price=transaction['transaction_price'],
                  type='Price'),
            total_consideration=models.CurrencyAndAmount(
              amount=transaction['transaction_cost'],
              currency=transaction['transaction_currency']),
             source='Default',
             transaction_currency=transaction['transaction_currency']))
        
    # Call LUSID to upsert your transactions for this portfolio
    response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transactions(
        scope=scope,
        code=portfolio,
        transaction_request=transactions)
    
    # Print the response from LUSID using pretty formatting 
    prettyprint.transactions_response(
        response,
        scope,
        portfolio)

Transactions Successfully Upserted into Portfolio
Scope: investment-book-of-records-3bf2-7ffd-98a6-44
Code: Global-Strategies
Transactions Effective From: 2023-06-20 16:38:55.568651+00:00
Transactions Created On: 2023-06-21 16:38:56.769105+00:00



![Transactions](img/migration-addtransactions.gif)

### 8) Retrieve your Holdings

You can see how these transactions have affected your holdings by trying to retrieve the holdings of your portfolio. By configuring your transaction types LUSID should have been able to decompose each transaction into its appropriate economic movements. 

For further usage of the get holdings API call refer to the [LUSID API Docs: Get Holdings](https://docs.lusid.com/#operation/GetHoldings).

*Run the cell below to get our holdings from our portfolio*

In [46]:
# Try and get your holdings, otherwise provide the error message
try:
    response = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
        scope=scope,
        code=portfolio_code,
        property_keys=['Instrument/default/Name'])
    prettyprint.holdings_response(response, scope, portfolio_code)
except lusid.rest.ApiException as e:
    print ('Error retrieving holdings. {}'.format(e))

Error retrieving holdings. (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Date': 'Wed, 21 Jun 2023 16:38:57 GMT', 'Content-Type': 'application/problem+json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'lusid-meta-success': 'False', 'lusid-meta-requestid': '0HMRIE84F6H4T:00000026', 'lusid-meta-correlationid': '0HMRIE84F6H4T:00000026', 'lusid-meta-duration': '709', 'x-envoy-upstream-service-time': '712', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'Server': 'FINBOURNE', 'Content-Security-Policy': "default-src 'self' https://*.lusid.com https://*.finbourne.com; script-src 'unsafe-inline' 'self' https://*.lusid.com https://*.finbourne.com https://editor.swagger.io; font-src 'self' fonts.googleapis.com; img-src data: 'self' https://*.lusid.com https://*.finbourne.com https://validator.swagger.io; style-src 'unsafe-inline' 'self' https://*.lusid.com https://*.finbourne.com; report-uri https://lusid.report-ur

Here you can see that it looks like you have a transaction type in your transactions file which is not in your configuration.

### 9) Update our Transaction Type Configuration

You can find out what might be the problematic transaction type by listing all of your unique transaction types.

*Run the cell below to find your unique transaction types*

In [47]:
# Get your unique list of transaction types
transaction_types = daily_transactions['transaction_type'].unique()
print (transaction_types)

['Sell' 'Buy' 'BrokerSell']


Here you can see that you have three transaction types in your list of transactions, you can check that they are all in your current configuration. You can do this by listing your current configuration in LUSID.

For further usage of the list configuration transaction types API call refer to the [LUSID API Docs: List Transaction Type Configuration](https://docs.lusid.com/#operation/ListConfigurationTransactionTypes).

*Run the cell below to check your transaction type configuration*

In [48]:
# Call LUSID to get your transaction type configuration
response = api_factory.build(lusid.api.SystemConfigurationApi).list_configuration_transaction_types()
# Pretty print the configuration
prettyprint.transaction_type_response(response, filters=['Buy', 'Sell', 'BrokerSell'])

Transaction Configuration #40

Transaction Type Aliases
Transaction Type: Buy
Alias Description: Purchase
Transaction Class: Basic
Transaction Group: default
Transaction Roles: LongLonger


Transaction Movements
Movement Types: StockMovement
Side: Side1
Direction: 1
Movement Types: CashCommitment
Side: Side2
Direction: -1



Transaction Configuration #41

Transaction Type Aliases
Transaction Type: Sell
Alias Description: Sale
Transaction Class: Basic
Transaction Group: default
Transaction Roles: LongShorter


Transaction Movements
Movement Types: StockMovement
Side: Side1
Direction: -1
Movement Types: CashCommitment
Side: Side2
Direction: 1



Transaction Configuration #82

Transaction Type Aliases
Transaction Type: BrokerSell
Alias Description: A sale of an equity via a broker
Transaction Class: Equity
Transaction Group: BrokerSystem
Transaction Roles: Shorter


Transaction Movements
Movement Types: StockMovement
Side: Side1
Direction: 1
Movement Types: CashCommitment
Side: Side2
Dire

Here you can see that although you have the 'Sell' and 'Buy' transaction types, you are missing the 'BrokerSell' transaction type. 

![Transactions](img/migration-missingconfig.gif)

You need to add this transaction type into your configuration so that LUSID can correctly deconstruct the transaction into its underlying economic movements. To do this you can take a look at the 'BrokerSell' transactions to see what they represent.

*Run the cell below to see all your BrokerSell transactions*

In [49]:
# Get all your transactions with the BrokerSell transaction type
brokersell_transactions = daily_transactions.loc[daily_transactions['transaction_type']=='BrokerSell']
brokersell_transactions.head()

,portfolio_code,transaction_id,instrument_name,transaction_description,transaction_type,transaction_units,transaction_price,transaction_currency,transaction_strategy,transaction_cost,figi
5,Global-Strategies,tid_124319009950924,USTreasury_2.00_2021,Equity Sale,BrokerSell,-28000,98.10,USD,incomeRequirements,2746800.00,BBG00FN3B5K8


Here you can see that it looks just like a sell transaction, you can add it into your configuration so that it increases your cash balance and reduces the number of units of your security. 

*Run the cell below to add your BrokerSell transaction type to the configuration*

In [50]:
# Call LUSID to create your transaction type configuration for the BrokerSell type
brokersell_type = models.TransactionConfigurationDataRequest(
    aliases=[
        models.TransactionConfigurationTypeAlias(
            type="BrokerSell",
            description="A sale of an equity via a broker",
            transaction_class="Equity",
            transaction_group="BrokerSystem",
            transaction_roles="Shorter",
        )
    ],
    movements=[
        models.TransactionConfigurationMovementDataRequest(
            movement_types="StockMovement",
            side="Side1",
            direction=-1,
            properties=None,
            mappings=None,
        ),
        models.TransactionConfigurationMovementDataRequest(
            movement_types="CashCommitment",
            side="Side2",
            direction=1,
            properties=None,
            mappings=None,
        ),
    ],
    properties=None,
)

upsert_brokersell_config = upsert_transaction_type_alias(api_factory, new_transaction_config=[brokersell_type])

You have added the BrokerSell transaction type under the 'BrokerSystem' transaction group. This allows you to group all your broker transaction types together and keep them seperate from your internal systems. To ensure that the transaction uses this group of types to resolve the underlying economic movements you can change the source parameter to 'BrokerSystem' by upserting the broker transaction again. 

*Run the cell below to re-upsert the transaction with the BrokerSystem source*

In [51]:
# Initialise a list to hold your transaction requests
transactions = []

# Iterate over the transactions in the portfolio
for index, transaction in brokersell_transactions.iterrows():

    # Append your request to the list
    transactions.append(models.TransactionRequest(
        transaction_id=transaction['transaction_id'],
        type=transaction['transaction_type'],
        instrument_identifiers={
            'Instrument/default/Figi': transaction['figi']
        },
        transaction_date=transaction_date.isoformat(),
        settlement_date=settlement_date.isoformat(),
        units=transaction['transaction_units'],
        transaction_price=models.TransactionPrice(
              price=transaction['transaction_price'],
              type='Price'),
        total_consideration=models.CurrencyAndAmount(
          amount=transaction['transaction_cost'],
          currency=transaction['transaction_currency']),
         source='BrokerSystem',
         transaction_currency=transaction['transaction_currency']))

# Call LUSID to upsert your transactions for this portfolio
response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transactions(
    scope=scope,
    code=portfolio,
    transaction_request=transactions)

# Print the response from LUSID using pretty formatting 
prettyprint.transactions_response(
    response,
    scope,
    portfolio)

Transactions Successfully Upserted into Portfolio
Scope: investment-book-of-records-3bf2-7ffd-98a6-44
Code: Global-Strategies
Transactions Effective From: 2023-06-20 16:38:55.568651+00:00
Transactions Created On: 2023-06-21 16:39:09.828301+00:00



![Transactions](img/migration-addedconfig.gif)

Now you can try again to generate the holdings from your portfolio.

*Run the cell below to try and retrieve your holdings again*

In [52]:
# Try and get your holdings, otherwise provide the error message
try:
    response = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
        scope=scope,
        code=portfolio_code,
        property_keys=['Instrument/default/Name'])
    prettyprint.holdings_response(
        response, 
        scope, 
        portfolio_code)
except lusid.exceptions.ApiException as e:
    print ('Error retrieving holdings as {}'.format(e))

Holdings for Portfolio
Scope: investment-book-of-records-3bf2-7ffd-98a6-44
Code: Global-Strategies



### 10) Adjust Holdings with your End of Day Position

Great, you are able to generate your holdings from LUSID! Now that you have succesfully added your transactions into LUSID, you can update your portfolio with the end of day position to ensure that your source system and LUSID are in sync.

You will import your end of day holdings from a CSV file.

*Run the cell below to import your end of day holdings*

In [53]:
end_of_day_holdings = pd.read_csv('data/demo-holdings2.csv')
end_of_day_holdings.head()

,portfolio_code,instrument_name,quantity,price,currency,figi
0,Global-Strategies,GBP_Cash,8336000,1.00,GBP,NaN
1,Global-Strategies,Glencore_LondonStockEx_GLEN,905141,2.76,GBP,BBG001MM1KV4
2,Global-Strategies,Kingfisher_LondonStockEx_KGF,1037038,2.28,GBP,BBG000BKH1W6
3,Global-Strategies,UKGiltTreasury_2.0_2025,405589,106.64,GBP,BBG0088JSC32
4,Global-Strategies,UKGiltTreasury_3.5_2045,266169,134.43,GBP,BBG006N6HZM7


Now that you have your holdings you can set your portfolio holdings just like you did at the start of the day to ensure that it matches the source system perfectly.

*Run the cell below to adjust your holdings to these levels*

In [54]:
# Set the holdings to be effective from half a day ago
holdings_effective_date = datetime.now(pytz.UTC) - timedelta(days=0.5)

# Iterate over the portfolios in our holdings, in this case only one
for portfolio in end_of_day_holdings['portfolio_code'].unique():
    
    # Initialise a list to hold your adjustments
    holding_adjustments = []
    
    # Iterate over the end of day holdings for this portfolio
    for index, holding in end_of_day_holdings.loc[end_of_day_holdings['portfolio_code'] == portfolio].iterrows():
        
        # Ensure that you use the correct identifier based on the instrument
        if 'Cash' in holding['instrument_name']:
            identifier_key = 'Instrument/default/Currency'
            identifer = holding['instrument_name'].split('_')[0]
        else:
            identifier_key = 'Instrument/default/Figi'
            identifer = holding['figi']
            
        # Build your adjustment and add it to your list
        holding_adjustments.append(
            models.AdjustHoldingRequest(
                instrument_identifiers={
                    identifier_key: identifer},
                tax_lots=[
                    models.TargetTaxLotRequest(
                        units=holding['quantity'],
                        cost=models.CurrencyAndAmount(
                            amount=holding['quantity'] * holding['price'],
                            currency=holding['currency']),
                        portfolio_cost=holding['quantity'] * holding['price'],
                        price=holding['price'])
                ]
            )
        )
        
    # Call LUSID to set your end of day position
    response = api_factory.build(lusid.api.TransactionPortfoliosApi).set_holdings(
        scope=scope,
        code=portfolio,
        effective_at=holdings_effective_date.isoformat(),
        adjust_holding_request=holding_adjustments)

    holdings_as_at = response.version.as_at_date

    # Pretty print the response from LUSID
    prettyprint.set_holdings_response(
        response, 
        scope, 
        portfolio)

Holdings Successfully Set for Portfolio
Scope: investment-book-of-records-3bf2-7ffd-98a6-44
Code: Global-Strategies
Holdings Effective From: 2020-08-03 16:38:35.884666+00:00
Holdings Created On: 2023-06-21 16:39:12.816871+00:00



![Transactions](img/migration-endofday.gif)

### 11) Bi-temporal Reconciliation

Now you can make use of LUSID's bi-temporal data store to compare your portfolio before and after adjusting for the end of day position. You can do this by performing a bi-temporal reconciliation. Read more about reconciliations here [LUSID Knowledge Base: Performing a Reconciliation](https://support.lusid.com/how-do-i-reconcile-portfolios-from-different).

By looking for reconciliation breaks between your portfolio before it was adjusted and after it was adjusted you can see what discrepancies have arisen from any issues with the transaction feed during the day. 

For further usage of the reconcile holdings API call refer to the [LUSID API Docs: Reconcile Holdings](https://docs.lusid.com/#operation/ReconcileValuation).

*Run the cell below to reconcile your portfolio with itself before and after the adjustment*

In [55]:
# Build your left side of the reconciliation before you made the adjustment
before_adjustment = models.PortfolioReconciliationRequest(
    portfolio_id=models.ResourceId(
        scope=scope,
        code=portfolio_code),
    effective_at=(holdings_effective_date - timedelta(hours=1)).isoformat(),
    as_at=holdings_as_at)

# Build your right side of the reconciliation after you made the adjustment
after_adjustment = models.PortfolioReconciliationRequest(
    portfolio_id=models.ResourceId(
        scope=scope,
        code=portfolio_code),
    effective_at=(holdings_effective_date + timedelta(hours=1)).isoformat(),
    as_at=holdings_as_at)

# Create your reconciliation request
reconcile_holdings_request = models.PortfoliosReconciliationRequest(
    left=before_adjustment,
    right=after_adjustment,
    instrument_property_keys=['Instrument/default/Name', 'Instrument/default/Figi'])

# Reconcile your two portfolios
response = api_factory.build(lusid.api.ReconciliationsApi).reconcile_holdings(
    portfolios_reconciliation_request=reconcile_holdings_request)

# Pretty print the response
prettyprint.reconciliation_response(
    response,
    scope,
    portfolio_code)

Reconciliation Breaks for Portfolio
Scope: investment-book-of-records-3bf2-7ffd-98a6-44
Code: Global-Strategies

Reconciliation Break
Instrument LUID: LUID_0000MIPE
Left Units: 1411244.0
Right Units: 1355244.0
Difference In Units: -56000.0
Left Cost: 138166387.6
Right Cost: 132678387.6
Difference In Cost: -5488000.0
Currency: USD
Instrument/default/Name: {'label_value': 'USTreasury_2.00_2021',
 'label_value_set': None,
 'metric_value': None}
Instrument/default/Figi: {'label_value': 'BBG00FN3B5K8', 'label_value_set': None, 'metric_value': None}




![Transactions](img/migration-reconciliation.gif)

### 12) Investigate Breaks

From this reconciliation you can see that there is a single break. You can find the transactions that may been responsible for this break by getting all transactions made against the instrument in the break.

For further usage of the get transactions API call refer to the [LUSID API Docs: Get Transactions](https://docs.lusid.com/#operation/GetTransactions).

*Run the cell below to find all transactions made against this instrument*

In [56]:
# Parse the Lusid Instrument Id from the reconciliation break
luid = response.values[0].instrument_uid

# Call LUSID to get your transactions made against this Lusid Instrument Id
response = api_factory.build(lusid.api.TransactionPortfoliosApi).get_transactions(
    scope=scope,
    code=portfolio_code,
    from_transaction_date=transaction_date,
    to_transaction_date=transaction_date,
    property_keys=['Instrument/default/Name'],
    filter="instrumentuid eq '" + luid + "'"
    )

# Pretty print the response
prettyprint.get_transactions_response(
    response,
    scope,
    portfolio_code,
    ['Instrument/default/Name'])

Transactions Retrieved from Portfolio
Scope:  investment-book-of-records-3bf2-7ffd-98a6-44
Code:  Global-Strategies 



,Transaction ID,Transaction Type,Instrument/default/Name,Units,Price,Currency,Transaction Date
0,tid_234942982496001,Sell,USTreasury_2.00_2021,57000.00,97.80,USD,2023-06-20 16:38:55.568651+00:00
1,tid_124319009950924,BrokerSell,USTreasury_2.00_2021,-28000.00,98.10,USD,2023-06-20 16:38:55.568651+00:00


Here you can see something interesting. Even though you have configured the transaction type for Sell and BrokerSell to have the same underlying economic movements, it looks like the units for a BrokerSell are passed in as a signed number whereas for the Sell the number of units is not signed. 

In fact the break is for 56000 units and our BrokerSell transaction is for 28000 units, exactly half the size of the break. Perhaps this is due to the sign on the units of the trade resulting in the number of this instrument being held moving in the wrong direction. 

You can fix this by updating the transaction configuration. In this case we need to adjust the direction of the 'Side1' movement from -1 to 1. This will take into account the BrokerSell transaction using a signed number for units.

*Run the cell below to update our transaction configuration*

In [57]:
# Add our updated BrokerSell configuration to our default configuration
brokersell_type = models.TransactionConfigurationDataRequest(
    aliases=[
        models.TransactionConfigurationTypeAlias(
            type="BrokerSell",
            description="A sale of an equity via a broker",
            transaction_class="Equity",
            transaction_group="BrokerSystem",
            transaction_roles="Shorter",
        )
    ],
    movements=[
        models.TransactionConfigurationMovementDataRequest(
            movement_types="StockMovement",
            side="Side1",
            direction=1,
            properties=None,
            mappings=None,
        ),
        models.TransactionConfigurationMovementDataRequest(
            movement_types="CashCommitment",
            side="Side2",
            direction=1,
            properties=None,
            mappings=None,
        ),
    ],
    properties=None,
)

upsert_brokersell_config = upsert_transaction_type_alias(api_factory, new_transaction_config=[brokersell_type])

So that you can easily check the last time that you updated your transaction type configuration, you can set a property on your portfolio with the date of the last change. Read more about properties here [LUSID Knowledge Base: Properties](https://support.lusid.com/what-is-a-property). 

For further usage of the create property definition API call refer to the [LUSID API Docs: Create Property Definition](https://docs.lusid.com/#operation/CreatePropertyDefinition).

*Run the cell below to create a property to hold the date of the last change*

In [58]:
# Create our request to define a new property
request = models.CreatePropertyDefinitionRequest(
    domain='Portfolio',
    scope=scope,
    code='lastconfigurationchange',
    value_required=False,
    display_name='lastconfigurationchange',
    data_type_id=models.ResourceId(scope='system', code='date'))

# Call LUSID to create our new property
response = api_factory.build(lusid.api.PropertyDefinitionsApi).create_property_definition(
    create_property_definition_request=request)

# Grab the key off the response to use when referencing this property in other LUSID calls
portfolio_property_key = response.key

# Pretty print our key
prettyprint.heading('Portfolio Property Key', portfolio_property_key)

Portfolio Property Key: Portfolio/investment-book-of-records-3bf2-7ffd-98a6-44/lastconfigurationchange


Now you can set the value of this property on your portfolio, as you just changed the configuration you can use the current datetime.

For further usage of the upsert portfolio properties API call refer to the [LUSID API Docs: Upsert Portfolio Properties](https://docs.lusid.com/#operation/UpsertPortfolioProperties).

*Run the cell below to add the date of the last change to your portfolio*

In [59]:
# Call LUSID to add the date of the last change to the portfolio
response = api_factory.build(lusid.api.PortfoliosApi).upsert_portfolio_properties(
    scope=scope,
    code=portfolio_code,
    request_body={
        portfolio_property_key: models.ModelProperty(
            key=portfolio_property_key,
            value=models.PropertyValue(label_value=datetime.now(pytz.UTC)))
    }
)

# Pretty print the response 
prettyprint.portfolio_properties_response(response)

Properties Sucessfully Updated for Portfolio
Property key: Portfolio/investment-book-of-records-3bf2-7ffd-98a6-44/lastconfigurationchange
Value: 2023-06-21T16:39:33.506184+00:00



![Portfolio Property](img/migration-portfolioproperty.gif)

## 13) Re-run Bi-temporal Reconciliation

You can now re-run the bi-temporal reconciliation to see if changing the transaction type configuration has removed the break. 

*Run the cell below to re-run the reconciliation to see if the break has been fixed*

In [60]:
# Build your left side of the reconciliation before you made the adjustment
before_adjustment = models.PortfolioReconciliationRequest(
    portfolio_id=models.ResourceId(
        scope=scope,
        code=portfolio_code),
    effective_at=(holdings_effective_date - timedelta(hours=1)).isoformat())

# Build your right side of the reconciliation after you made the adjustment
after_adjustment = models.PortfolioReconciliationRequest(
    portfolio_id=models.ResourceId(
        scope=scope,
        code=portfolio_code),
    effective_at=(holdings_effective_date + timedelta(hours=1)).isoformat())

# Create your reconciliation request
reconcile_holdings_request = models.PortfoliosReconciliationRequest(
    left=before_adjustment,
    right=after_adjustment,
    instrument_property_keys=['Instrument/default/Name', 'Instrument/default/Figi'])

# Reconcile your two portfolios
response = api_factory.build(lusid.api.ReconciliationsApi).reconcile_holdings(
    portfolios_reconciliation_request=reconcile_holdings_request
)

# Pretty print the response
prettyprint.reconciliation_response(
    response,
    scope,
    portfolio_code)

Reconciliation Breaks for Portfolio
Scope: investment-book-of-records-3bf2-7ffd-98a6-44
Code: Global-Strategies

No Reconciliation Breaks


Great you no longer have any breaks and your transaction feed for the day is in-sync with the source system. You can now repeat this process as more transactions come in over the coming days.